In [ ]:
import pcraster as pcr
pcr.setrandomseed(8)
buildgMap = pcr.readmap("mapalgebra/buildg.map")
firestatMap = pcr.readmap("mapalgebra/firestat.map")
iswaterMap = pcr.readmap("mapalgebra/iswater.map")
phreaticMap = pcr.readmap("mapalgebra/phreatic.map")
rainstorMap = pcr.readmap("mapalgebra/rainstor.map")
roadsMap = pcr.readmap("mapalgebra/roads.map")
topoMap = pcr.readmap("mapalgebra/topo.map")
waterMap = pcr.readmap("mapalgebra/water.map")
dumpMap = pcr.readmap("mapalgebra/dump.map")
isroadMap = pcr.readmap("mapalgebra/isroad.map")
loggingMap = pcr.readmap("mapalgebra/logging.map")
pointsMap = pcr.readmap("mapalgebra/points.map")
rainyearMap = pcr.readmap("mapalgebra/rainyear.map")
soilsMap = pcr.readmap("mapalgebra/soils.map")
treesMap = pcr.readmap("mapalgebra/trees.map")
wellsMap = pcr.readmap("mapalgebra/wells.map")

Download the dataset here.

In this lab you will calibrate parameters of a hydrological model for the Dorfertal catchment near Kals (Austria). Use aguila to display the digital elevation map (dem.map, elevation above sea level in metres) the local drain direction map (ldd.map) and the location of the streamflow measurements (sample_location.map). The cell size used is 100 m. DEM data are from Open Data Oesterreich.

As you can see on the photo below, even in summer the peaks are covered with snow.

Timeseries data as input to the model are plotted in observed_timeseries.pdf. Temperature and precipitation are taken from CFSR, streamflow from GRDC.

<blockquote>
<div><figure class="align-left" id="id1" style="width: 50%">
<img alt="../_images/dorfertal.jpg" src="../_images/dorfertal.jpg"/>
<figcaption>
<p><span class="caption-text">Dorfertal catchment with main stream, photo taken in summer.</span><a class="headerlink" href="#id1" title="Link to this image">¶</a></p>
</figcaption>
</figure>
</div></blockquote>

Except interception of precipitation (i.e., rainfall plus snowfall) by the vegetation, the model represents all main components of a hydrological system, where each component is modelled using simple equations. The figure below shows the model structure. The model has two storages, the snow storage and the subsurface storage. The latter represents all the water that is stored below the surface. The input of water is precipitation, the outputs from the system are streamflow and loss of water to the atmosphere. A number of fluxes is defined each defined by a particular equation. These equations use so called parameters, which are mostly assumed to be fixed. They are shown in red in the schematic. It is often hard to directly measure parameters and this is why they are ‘calibrated’. We will come back to this later.

An essential component in the model is the air temperature. As it is not measured for all pixels in the area, we use a time series of temperature observed at a certain reference elevation. As temperature mostly decreases with elevation, temperature is then estimated for each pixel using the digital elevation model (surface level of each pixel). For detailed model equations refer to the text below. Most important in this exercise is the snowmelt rate parameter  which is the one that will be calibrated.

<blockquote>
<div><figure class="align-left" id="id2" style="width: 100%">
<img alt="../_images/schematic.jpg" src="../_images/schematic.jpg"/>
<figcaption>
<p><span class="caption-text">Schematic of the model. Reference elevation of temperature is in principle given by the dataset, but has some uncertainty and could be adjusted in calibration, which is why it is shown as a parameter here.</span><a class="headerlink" href="#id2" title="Link to this image">¶</a></p>
</figcaption>
</figure>
</div></blockquote>

The equations below are evaluated for each grid cell, for all timesteps (time step is 1 day):

The air temperature (, degrees Celcius) above the surface is:

With, , the observed temperature (degrees Celcius); , the temperature lapse rate (degrees / m elevation); , the elevation of the land surface; , the reference elevation of the observed temperature.

Snowfall (, m equivalent water depth/day) is

With, , the observed precipitation (m/day).

Rainfall (, m/day) is

The rate of change in the snow depth (, m equivalent water depth) is

With, , meltrate parameter (degree day factor, m/(day degree)); , sublimation (m/day). The sublimation is equal to the loss of water to the atmosphere parameter (, m/day); if the snow depth  becomes zero, sublimation becomes zero and loss of water to the atmosphere is taken from the subsurface water storage (see below).

The water flux available for infiltration () equals the sum of snowmelt and rainfall:

With, , the snow melt rate parameter (m/(day degree)). The actual infiltration (, m/day) is:

With,  the infiltration capacity parameter (m/day).

Water in the subsurface is represented by  (m), this is the amount of water stored below the surface, loosely defined as groundwater. Upward seepage from the subsurface (, m/day, loss of groundwater to the surface water) is:

With, , seepage parameter (/day). The rate of change in the groundwater is:

With,  the loss to the atmosphere which is zero in case of snow cover (in which case loss to the atmosphere is taken from the snow cover).

The amount of surface runoff (m/day) generated is:

The streamflow is calculated for each day as the sum of  in all the upstream pixels of a cell.

Inspect the script runoff.py. It is a standard PCRaster Python script except for the added capability to pass a parameter value to the actual model; the def __init__ statement takes an argument meltRateParameter which is passed when calling the model at the bottom of the script. At the bottom of the script is also some code that will be used for the calibration, which will be explained later on. The model simulates the same time span as given by observed_timeseries.pdf, i.e. three years.

Execute the model. Inspect the timeseries of maps created by the model; snow depth, snow (snow0000.001, snow0000.002, etc maps); subsurface storage, sub; and streamflow discharge, dis. For instance:

In [ ]:
import subprocess 
subprocess.call("""aguila dem.map sample_location.map --timesteps=[1,1461,1] snow dis
""", shell=True)

Do not forget to right-mouse click on the legend to plot the timeseries - if you click on the map you can then plot the timeseries for any location.

Question 1: Does the snowdepth at timestep 1 have a realistic value?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>Yes, it is midsummer and snowdepth is thus close to zero.</p></li>
<li><p>Yes, it is midwinter but there was hardly any snowfall resulting in zero snowdepths.</p></li>
<li><p>No, it is assumed snowdepth is zero at the start as there is no information on the starting value.</p></li>
<li><p>No, it is assumed snowdepth is at a maximum at the start as there is no information on the starting value.</p></li>
</ol>
</div></blockquote>

Question 2: What is the mechanism determining larger amounts of streamflow in spring?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>Snowmelt. The snowdepth is decreasing in spring causing high streamflow. In addition, peaks in streamflow occur due to direct rain.</p></li>
<li><p>Rainfall. The rainfall is seasonal with highest rain in winter and spring. In winter it falls as snow but in spring it falls as rain causing high streamflow.</p></li>
<li><p>Loss of water to the atmosphere. This changes over time, with highest losses in summer and winter, causing higher streamflow in spring (and autumn).</p></li>
<li><p>Wrong! Streamflow is highest in summer when precipitation falls as rain, which drains to the stream.</p></li>
</ol>
</div></blockquote>

Let’s now compared modelled streamflow with observed (measured) streamflow. Here we do so by importing the observed streamflow (at the measurement location) to the model and reporting it as a map. This is somewhat cumbersome but has the advantage that we can evaluate results with aguila.

Add the following to the bottom of the dynamic section:

In [ ]:
# read observed discharge
dis_obs = pcr.timeinputscalar("streamflow.txt", pcr.ifthen(self.sampleLocation, pcr.boolean(1)))


And add additional statements that:

<ul class="simple">
<li><p>Calculate the difference between modelled and observed streamflow (use the <code class="docutils literal notranslate"><span class="pre">-</span></code> operator, modelled minus observed).</p></li>
<li><p>Write this difference map to disk (<code class="docutils literal notranslate"><span class="pre">report</span></code>), use <code class="docutils literal notranslate"><span class="pre">dMmO</span></code> as output file name.</p></li>
<li><p>Write the observed discharge to disk (<code class="docutils literal notranslate"><span class="pre">dis_obs</span></code>), use <code class="docutils literal notranslate"><span class="pre">dO</span></code> as output file name.</p></li>
</ul>

Run the model and display dO, dMmO, and dis. Right-click on the legend to plot the timeseries and then click on the measurement location to retrieve the values at the this location in the timeseries.

Question 3: What is the main difference between modelled and observed streamflow?

<blockquote>
<div><ol class="loweralpha simple">
<li><p>The streamflow is overestimated in midwinter.</p></li>
<li><p>The streamflow is often too high in autumn.</p></li>
<li><p>The streamflow is often too low in spring and too high in summer.</p></li>
<li><p>The streamflow is often too high in spring and too low in summer.</p></li>
</ol>
</div></blockquote>